# Notebook for processing Landsat imagery for deep learning classification

### Import packages

In [4]:
import sys, os
import re
import arcpy

arcpy.env.parallelProcessingFactor = "100%"
arcpy.env.addOutputsToMap = False
arcpy.env.overwriteOutput = True
arcpy.env.processorType = "GPU"

### Set parameters and options

In [5]:
# Location of input imagery
in_folder = r"G:\My Drive\Agdam_Landsat_Max_Filled2"
if arcpy.Exists(in_folder) != True: print("Bad Folder")
    
# Holding/scratch folder (will be created by ArcGIS when needed)
holding = r"C:\Users\Neal\Documents\ArcGIS\Projects\NKR4\delete"

# Change value to True if clipping needed, else False
clip_selector = False
clipping_extent = "649947.8909 4410578.6392 714505.2748 4475136.0231"

# Set folders for models
training_raster = "ag_cmp_2008.png"
train_folder = r"C:\Users\Neal\Documents\ArcGIS\Projects\NKR4\DL Models\Train Data"
model_folder = r"C:\Users\Neal\Documents\ArcGIS\Projects\NKR4\DL Models\Trained Model"
classified_polygon = r"C:\Users\Neal\Documents\ArcGIS\Projects\NKR4\NKR4.gdb\c1984_reclass_poly"
classified_mask = r"C:\Users\Neal\Documents\ArcGIS\Projects\NKR4\NKR4.gdb\mask2_Dissolve_Smooth"

### Create folder names and directories

In [ ]:
clipped_renamed_folder =  os.path.join(holding,"clipped")
try: os.mkdir(clipped_renamed_folder)
except FileExistsError: print(f"{clipped_renamed_folder} already exists.")
    
layers_folder = os.path.join(holding,'Raster Layers')
try: os.mkdir(layers_folder)
except FileExistsError: print(f"{layers_folder} already exists.")

output_composite_folder = os.path.join(in_folder,"output_composite")
try: os.mkdir(output_composite_folder)
except FileExistsError: print(f"{output_composite_folder} already exists.")

output_ms_folder = os.path.join(in_folder,"output_ms")
try: os.mkdir(output_ms_folder)
except FileExistsError: print(f"{output_ms_folder} already exists")

# output_cir_folder = os.path.join(in_folder,"output_cir")
# try: os.mkdir(output_cir_folder)
# except FileExistsError: print(f"{output_cir_folder} already exists.")

# output_ndvi_folder = os.path.join(in_folder,"output_ndvi")
# try: os.mkdir(output_ndvi_folder)
# except FileExistsError: print(f"{output_ndvi_folder} already exists.")

# output_rns_folder = os.path.join(in_folder,"output_rns")
# try: os.mkdir(output_rns_folder)
# except FileExistsError: print(f"{output_rns_folder} already exists.")

output_classified_folder = os.path.join(in_folder,"output_class")
try: os.mkdir(output_classified_folder)
except FileExistsError: print(f"{output_classified_folder} already exists")

## Prepare imagery for classification

### Gather rasters and either: Clip to set extent (and rename),or just Rename if clipping is not necessary, then add to GDB

In [ ]:
arcpy.env.workspace = in_folder
rasters = arcpy.ListRasters()
rasters.sort()

for raster in rasters:
    desc = arcpy.Describe(raster)
    year_search = re.search(r'\d\d\d\d',desc.baseName)
    year = year_search.group()
    if year == None:
        print('No year value found in name.')
        sys.exit(1)
    
    try:
        if clip_selector == True:
            arcpy.management.Clip(raster, clipping_extent, 
                    os.path.join(clipped_renamed_folder,f"ag_c_{year}"))
            print(f"Clipped {desc.baseName}")
            intermediate_folder = clipped_folder
        else:
            if len(desc.baseName) > 9:
                arcpy.management.Rename(raster, f"ag_c_{year}")
                print(f"Renamed {desc.baseName}")
            intermediate_folder = in_folder
            
    except Exception as e:
        print(e)

# Add rasters to geodatabase/folder as Raster Datasets
arcpy.env.workspace = intermediate_folder
renamed_clipped_rasters = arcpy.ListRasters()
renamed_clipped_rasters.sort()
                              
for raster in renamed_clipped_rasters:
    desc = arcpy.Describe(raster)
    if len(desc.baseName) < 10:
        try:
            arcpy.RasterToGeodatabase_conversion(raster,layers_folder)
            print(f"Raster {desc.baseName} to Feature Layer")
        except Exception as e:
            print(e)
    else:
        print('Need to shorten file names before adding to GDB')

### Generate desired imagery as PNGs, and move to output folders   

In [ ]:
arcpy.env.workspace = layers_folder
feature_layers = arcpy.ListRasters()

for layer in feature_layers:
    desc = arcpy.Describe(layer)
    if len(desc.baseName) < 10:
        print("Working on:",desc.baseName)
        
        # Uncomment lines to change type of image generated for classification
        # Currently set to just the NIR/SWIR/PC1 image (CMP) and mean-shift (MS)
        try:     
            img_name = f"ag_cmp_{desc.baseName[-4:]}.png"
            nir_raster = arcpy.management.MakeRasterLayer(layer, "nir_temp", '', "", "4")
            swir_raster = arcpy.management.MakeRasterLayer(layer, "swir_temp", '', "", "5")
            pc1_raster = arcpy.sa.PrincipalComponents(layer, 1, None) 
            arcpy.management.CompositeBands(["nir_temp","swir_temp",pc1_raster],'temp_img')
            arcpy.management.CopyRaster('temp_img', os.path.join(output_composite_folder, img_name),
                    '',None, "3.4e+38", "NONE", "NONE", '16_BIT_UNSIGNED', "ScalePixelValue",
                    "NONE", "PNG", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

            ms_name = f"ag_ms_{desc.baseName[-4:]}.png"
            mean_shift = arcpy.sa.SegmentMeanShift(layer, "18", "18", "25", "4 3 2")
            print(f"Segment Mean Shift on {ms_name} completed.")
            arcpy.management.CopyRaster(mean_shift, os.path.join(output_ms_folder, ms_name),
            '',None, "3.4e+38", "NONE", "NONE", '16_BIT_UNSIGNED', "ScalePixelValue",
            "NONE", "PNG", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")
            print(f"Output for {ms_name} complete")
            
#             rns_name = f"ag_rns_{desc.baseName[-4:]}.png"
#             arcpy.management.MakeRasterLayer(layer, rns_name, '', "", "4;3;5")
#             arcpy.management.CopyRaster(rns_name, os.path.join(output_rns_folder, rns_name),
#                 '',None, "3.4e+38", "NONE", "NONE", '16_BIT_UNSIGNED', "NONE",
#                 "NONE", "PNG", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")
#             print(f"Red_NIR_SWIR1 for {rns_name} complete.")
          
#             cir_name = f"ag_cir_{desc.baseName[-4:]}.png"
#             arcpy.management.MakeRasterLayer(layer, cir_name, '', "", "4;3;2")
#             arcpy.management.CopyRaster(cir_name, os.path.join(output_cir_folder, cir_name),
#                 '',None, "3.4e+38", "NONE", "NONE", '16_BIT_UNSIGNED', "ScalePixelValue",
#                 "NONE", "PNG", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")
#             print(f"Color Infrared for {cir_name} complete.")
            
#             ndvi_name = f"ag_ndvi_{desc.baseName[-4:]}.png"
#             arcpy.management.MakeRasterLayer(layer, ndvi_name, '', "", "7")
#             arcpy.management.CopyRaster(ndvi_name, os.path.join(output_ndvi_folder, ndvi_name),
#                 '',None, "3.4e+38", "NONE", "NONE", '16_BIT_UNSIGNED', "ScalePixelValue",
#                 "NONE", "PNG", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")
#             print(f"NDVI for {ndvi_name} complete.")

        except Exception as e:
            print(e)

## Prepare imagery for deep learning

### Set folders for imagery

In [7]:
arcpy.env.workspace = output_ms_folder
ms_rasters = arcpy.ListRasters()

arcpy.env.workspace = output_composite_folder
rns_rasters = arcpy.ListRasters()
rns_rasters.sort()

arcpy.env.workspace = output_composite_folder
cmp_rasters = arcpy.ListRasters()
cmp_rasters.sort()

### Generate an unsupervised classification image.
#### Assists in creating a classified polygon for supervised classification on the dataset

In [ ]:
arcpy.env.workspace = output_composite_folder

mean_shift_raster = ms_rasters[23]
unsupervised_image = arcpy.sa.IsoClusterUnsupervisedClassification(training_raster,25,50,50)
unsupervised_image.save(os.path.join(output_composite_folder,"unsup_img"))

# Edit the 'unsup_img' with expert-knowledge to correct labels before continuing

### Create image chips from the unsupervised classification

In [9]:
arcpy.env.workspace = output_composite_folder
arcpy.ia.ExportTrainingDataForDeepLearning(os.path.join(output_composite_folder,training_raster), train_folder, classified_polygon, 
                                           "PNG", 128, 128, 64, 64, "ONLY_TILES_WITH_FEATURES", 
                                           "Classified_Tiles", 0, "gridcode", 0, classified_mask, 0, 
                                           "MAP_SPACE", "PROCESS_AS_MOSAICKED_IMAGE", "NO_BLACKEN", "FIXED_SIZE")

### Train deep learning model

In [25]:
arcpy.ia.TrainDeepLearningModel(train_folder, model_folder, 10, "UNET", 16,
                                    "class_balancing False;mixup False;focal_loss False;ignore_classes #;chip_size 128",
                                    None, "RESNET18", None, 10, "STOP_TRAINING", "FREEZE_MODEL")

epoch     train_loss  valid_loss  accuracy  time    
0         0.572491    0.500950    0.744702  00:15     
1         0.446532    0.374969    0.835007  00:14     
2         0.393922    0.312092    0.867312  00:14     
3         0.384647    0.288390    0.873012  00:14     
4         0.371646    0.328187    0.848173  00:14     
5         0.342783    0.284472    0.873135  00:14     
6         0.330219    0.276041    0.879803  00:14     
7         0.329032    0.278885    0.880179  00:14     
8         0.326725    0.285471    0.872890  00:14     
9         0.320722    0.271949    0.882372  00:14     


<geoprocessing server result object object at 0x000001BAC6FDD810>

### Classify each raster in the collection and post-process

In [ ]:
for raster in rns_rasters:
    arcpy.env.workspace = output_composite_folder
    desc = arcpy.Describe(raster)
    out_classified_raster = arcpy.ia.ClassifyPixelsUsingDeepLearning(os.path.join(output_composite_folder,raster), 
                                             os.path.join(model_folder,"Trained Model.dlpk"), 
                                             "padding 16;batch_size 8;predict_background False", 
                                             "PROCESS_AS_MOSAICKED_IMAGE", None)

    with arcpy.EnvManager(outputZFlag="Disabled", outputMFlag="Disabled", workspace=holding):
        masked_raster = arcpy.sa.ExtractByMask(out_classified_raster, classified_mask)
        con_raster = arcpy.sa.Con(masked_raster, 1, None, "Class = '1'")
        arcpy.conversion.RasterToPolygon(con_raster, os.path.join(holding,f"poly{desc.baseName[-4:]}"), "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)
        arcpy.analysis.Buffer(os.path.join(holding,f"poly{desc.baseName[-4:]}.shp"),f"buffer{desc.baseName[-4:]}","15 Meters", "FULL", "ROUND", "NONE", None, "GEODESIC")
        arcpy.cartography.SmoothPolygon(f"buffer{desc.baseName[-4:]}.shp", os.path.join(output_classified_folder,f"cls_{desc.baseName[-4:]}"), 
                    "PAEK", "150 Meters", "FIXED_ENDPOINT", "NO_CHECK", None)